In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier


def pegar_dataframe_por_num_resposta(num_resposta: int):
    return pd.read_csv('perguntas_2019 - {} respostas - Pronto para uso (sem bias).csv'.format(num_resposta))

In [2]:
for num_respostas in range(3, 6):
    print("\n\n - - - - - Começando teste com {} respostas - - - - - \n\n".format(num_respostas))
    df = pegar_dataframe_por_num_resposta(num_respostas)

    # Filtrando apenas com perguntas com resposta escolhida
    df = df[df['alvo']!=-1]

    # O atributo 'tem_site_proprio' foi removido, pois não parecia estar contribuindo
    colunas_respostas = [ 'pontos_resposta', 'tempo_resposta', 'user_reputation', 'maestria_tags']
    colunas_relevantes = [c + str(resposta) for resposta in range(0, num_respostas) for c in colunas_respostas]

    x = df[colunas_relevantes]
    y = df['alvo']

    print('Tamanho dataset: ' + str(len(df)))

    # Preparamos o imputer
    simple_imputer = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=-1)
    simple_imputer.fit(x)
    x = simple_imputer.transform(x)

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

    # Dessa vez realizamos o scalling!
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    print('Numero linhas Treino: ' + str(len(X_train)))
    print('Numero linhas Test: ' + str(len(X_test)))

    model = SGDClassifier()
    model.fit(X_train, y_train)


    y_pred_train = model.predict(X_train)

    cm_treinamento = confusion_matrix(y_train, y_pred_train)

    print('\nConjunto de treinamento:\n')
    print('Matriz Confusão:\n' + str(cm_treinamento) + "\n")
    print(classification_report(y_train, y_pred_train))

    y_pred = model.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    print('\nConjunto de testes:\n')
    print('Matriz Confusão:\n' + str(cm) + "\n")
    print(classification_report(y_test, y_pred))

    print("\n\n - - - - - Terminado teste com {} respostas - - - - - \n\n".format(num_respostas))




 - - - - - Começando teste com 3 respostas - - - - - 


Tamanho dataset: 268680
Numero linhas Treino: 228378
Numero linhas Test: 40302

Conjunto de treinamento:

Matriz Confusão:
[[50433 10010 15416]
 [13945 46784 15514]
 [ 9359  9311 57606]]

              precision    recall  f1-score   support

           0       0.68      0.66      0.67     75859
           1       0.71      0.61      0.66     76243
           2       0.65      0.76      0.70     76276

    accuracy                           0.68    228378
   macro avg       0.68      0.68      0.68    228378
weighted avg       0.68      0.68      0.68    228378


Conjunto de testes:

Matriz Confusão:
[[ 9008  1696  2839]
 [ 2469  8247  2662]
 [ 1605  1644 10132]]

              precision    recall  f1-score   support

           0       0.69      0.67      0.68     13543
           1       0.71      0.62      0.66     13378
           2       0.65      0.76      0.70     13381

    accuracy                           0.68     403